## Example 3. Kepler's Two-Body Problem with Three Invariants

This notebook solves Kepler's Two-Body problem with three invariants by different RK methods and their relaxation versions:

\begin{align}
    \dot{q_{1}} & = p_{1} \\
    \dot{q_{2}} & = p_{2} \\
    \dot{p_{1}} & = -\frac{q_{1}}{\left(q_{1}^2+q_{2}^2\right)^{3/2}} \\
    \dot{p_{2}} & =-\frac{q_{2}}{\left(q_{1}^2+q_{2}^2\right)^{3/2}} \;,
\end{align}
with an initial condition $ \left(q_{1}(0),q_{2}(0),p_{1}(0),p_{2}(0)\right)^{T}=\left(1-e,0,0,\sqrt{\frac{1+e}{1-e}}\right)^{T}$, where $e=0.5$.

Three conserved quantities:

\begin{align}
    H(q,p) & = \frac{1}{2}\left(p_{1}^2+p_{2}^2\right)-\frac{1}{\sqrt{q_{1}^2+q_{2}^2}} \  \\
    L(q,p) & = q_{1}p_{2}-q_{2}p_{1} \\
    A(q,p) & = \textrm{$L_2$ norm of Laplace–Runge–Lenz vector} \;
\end{align}

 where in the last invariant, the well-known Laplace–Runge–Lenz vector function is defined as 
  \begin{align}
        \begin{bmatrix}
        p_{1} \\
        p_{2} \\
        0
        \end{bmatrix} \times \begin{bmatrix}
        0 \\
        0 \\
        q_{1} p_{2}-q_{2} p_{1}
        \end{bmatrix}-\frac{1}{\sqrt{q_{1}^{2}+q_{2}^{2}}} \begin{bmatrix}
        q_{1} \\
        q_{2} \\
        0
        \end{bmatrix}\;.
   \end{align}

In [ ]:
#Required libraries 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from nodepy import rk
from IPython.display import clear_output
from RKSchemes import ssp22, heun33, ssp33, rk44, fehlberg45, dp75

In [ ]:
# Required functions for Kepler's problem
def kepler_f(u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)   
    dq1 = p1
    dq2 = p2
    dp1 = -q1 / (abs_q*abs_q*abs_q)
    dp2 = -q2 / (abs_q*abs_q*abs_q)
    return np.array([dq1, dq2, dp1, dp2])

def kepler_energy_H(u):
    abs2_q = u[0]*u[0] + u[1]*u[1]
    abs2_p = u[2]*u[2] + u[3]*u[3]
    return 0.5 * abs2_p - 1.0 / np.sqrt(abs2_q)

def kepler_angular_momentum_L(u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    return q1*p2 - q2*p1

def kepler_LRL(u):
    q1 = u[0]
    q2 = u[1]
    p1 = u[2]
    p2 = u[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)
    A1 = p2*(q1*p2 - q2*p1) - q1/abs_q; A2 = -p1*(q1*p2 - q2*p1) - q2/abs_q
    return np.linalg.norm(np.array([A1,A2]))**1

def rgam(gammas,u,inc1,inc2,inc3,E1_old,E2_old,E3_old):
    gamma1, gamma2, gamma3 = gammas
    uprop = u + gamma1*inc1 + gamma2*inc2+ gamma3*inc3  
    E1 = kepler_energy_H(uprop)
    E2 = kepler_angular_momentum_L(uprop)
    E3 = kepler_LRL(uprop)
    return np.array([E1-E1_old,E2-E2_old,E3-E3_old])

"""
Analytical solution of the Kepler problem, cf.
http://mathworld.wolfram.com/KeplersEquation.html
and
https://matlab-monkey.com/astro/keplerEquation/KeplerEquationPub.html
"""
def u_kepler(t):
    e = 0.5
    u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])
    energy = 0.5 * (u0[2]*u0[2] + u0[3]*u0[3]) - 1.0 / np.sqrt(u0[0]*u0[0] + u0[1]*u0[1])
    momentum = u0[0]*u0[3] - u0[1]*u0[2]
    
    res = fsolve(lambda alpha: alpha - e*np.sin(alpha) - t, 0, xtol=1.e-10)
    alpha = res[0]
    theta = 2.0 * np.arctan(np.sqrt((1+e)/(1-e)) * np.tan(0.5*alpha))
    radius = (1 - e*e) / (1 + e * np.cos(theta))
    q0 = radius * np.cos(theta)
    q1 = radius * np.sin(theta)
    
    # using the conservation of angular momentum and energy
    abs2p = 2 * (energy + 1.0 / np.sqrt(q0*q0 + q1*q1))
    if q1 <= 0:
        p0 = -((momentum*q1 + np.sqrt(q0**2 * (-momentum**2 + abs2p*(q0**2 + q1**2)))) / (q0**2 + q1**2))
        p1 = (momentum*q0**2 - q1*np.sqrt(q0**2 * (-momentum**2 + abs2p*q0**2 + abs2p*q1**2))) / (q0**3 + q0*q1**2)
    else:
        p0 = (-(momentum*q1) + np.sqrt(q0**2 * (-momentum**2 + abs2p*(q0**2 + q1**2)))) / (q0**2 + q1**2)
        p1 = (momentum*q0**2 + q1 * np.sqrt(q0**2 * (-momentum**2 + abs2p*q0**2 + abs2p*q1**2))) / (q0**3 + q0*q1**2)
    return np.array([q0, q1, p0, p1])

### Baseline RK methods

In [ ]:
# Compute solution with baseline methods
def compute_sol_without_relaxation(Mthdname,rkm, dt, f, T, u0,t0): 
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
        inc = dt*sum([rkm.b[i]*F[i] for i in range(s)])    
        unew = uu[-1]+inc; t+= dt
        tt = np.append(tt, t)
        steps +=1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        print("Method = Baseline %s: Step number = %d (time = %1.2f)"%(Mthdname,steps,tt[-1]))
    return tt, uu

### Relaxation RK methods

In [ ]:
# Computing solution with multiple relaxation methods
def compute_sol_multi_relaxation(Mthdname,rkm, dt, f, T, u0, t0):
    tt = np.zeros(1) 
    t = 0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    
    G1 = np.array([]); G2 = np.array([]) ; G3 = np.array([]) 
    no_inv = 3; gammas0 = np.zeros(no_inv)
    
    errs = 0; steps = 0
    E10 = kepler_energy_H(u0); E20 = kepler_angular_momentum_L(u0); E30 = kepler_LRL(u0)
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
            
        inc1 = dt*sum([rkm.b[i]*F[i] for i in range(s)])
        inc2 = dt*sum([rkm.bhat[i]*F[i] for i in range(s)])
        inc3 = dt*sum([rkm.b3[i]*F[i] for i in range(s)])
        
        wr_unew = uu[-1] + inc1
        E1_old = kepler_energy_H(uu[-1]); E2_old = kepler_angular_momentum_L(uu[-1]); E3_old = kepler_LRL(uu[-1])
        
        gammas, info, ier, mesg = fsolve(rgam,gammas0,args=(wr_unew,inc1,inc2,inc3,E1_old,E2_old,E3_old),full_output=True)
        gamma1, gamma2, gamma3 = gammas
             
        unew =  wr_unew + gamma1*inc1 + gamma2*inc2+ gamma3*inc3; t+=(1+gamma1+gamma2+gamma3)*dt
        tt = np.append(tt, t); steps += 1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0) 
        G1 = np.append(G1, gamma1); G2 = np.append(G2, gamma2); G3 = np.append(G3, gamma3)
        print("Method = Relaxation %s: At step number = %d (time = %1.2f), integer flag = %d and γ1+γ2+γ3 = %f \n"%(Mthdname,steps,tt[-1],ier,gamma1+gamma2+gamma3))
    return tt, uu, G1, G2, G3

### Compute solution by all methods

In [ ]:
# Computing solutions by all the methods
%time 
eqn = 'Kep'
methods = [ssp33,fehlberg45,dp75]
method_labels = ["SSPRK(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
method_names = ["SSPRKs3p3", "Fehlbergs6p4", "DP5s7p5"]

# Inputs to solve the system of ODEs
DT = [.05, .05, .1];  f = kepler_f; T = 200;

# Initial condition
t0 = 0; 
e = 0.5; u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])

b_tt = []; b_uu = []; r_tt = []; r_uu = []  # empty list to store data by all the methods

# Relaxation methods
for idx in range(len(methods)):
    print(idx)
    rkm = methods[idx]; dt = DT[idx]
    tt, uu, G1, G2, G3 = compute_sol_multi_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    r_tt.append(tt); r_uu.append(uu) 
    
# Baseline methods
for idx in range(len(methods)):
    rkm = methods[idx]; dt = DT[idx]
    tt, uu = compute_sol_without_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    b_tt.append(tt); b_uu.append(uu)
    
   

In [ ]:
import os
path = '%s'%('Figures')

import os
if not os.path.exists(path):
   os.makedirs(path)

### Compute and plot change in invariants by different methods

In [ ]:
b_INV_H = []; b_INV_L = []; b_INV_LRL = []; 
r_INV_H = []; r_INV_L = []; r_INV_LRL = []; 
for i in range(len(methods)):
    b_inv_H = [kepler_energy_H(u) for u in b_uu[i]] - kepler_energy_H(b_uu[i][0])
    b_inv_L = [kepler_angular_momentum_L(u) for u in b_uu[i]] - kepler_angular_momentum_L(b_uu[i][0])
    b_inv_LRL = [kepler_LRL(u) for u in b_uu[i]] - kepler_LRL(b_uu[i][0])
    r_inv_H = [kepler_energy_H(u) for u in r_uu[i]] - kepler_energy_H(r_uu[i][0])
    r_inv_L = [kepler_angular_momentum_L(u) for u in r_uu[i]] - kepler_angular_momentum_L(r_uu[i][0])
    r_inv_LRL = [kepler_LRL(u) for u in r_uu[i]] - kepler_LRL(r_uu[i][0])
    b_INV_H.append(b_inv_H); b_INV_L.append(b_inv_L); b_INV_LRL.append(b_inv_LRL)
    r_INV_H.append(r_inv_H); r_INV_L.append(r_inv_L); r_INV_LRL.append(r_inv_LRL)

In [ ]:
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 
plt.figure(figsize=(15, 4))
for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i],b_INV_H[i],':r',label="Baseline: $H(p(t),q(t))-H(p(0),q(0))$")
    plt.plot(r_tt[i],r_INV_H[i],'-r',label="Relaxation: $H(p(t),q(t))-H(p(0),q(0))$")
    plt.plot(b_tt[i],b_INV_L[i],':b',label="Baseline: $L(p(t),q(t))-L(p(0),q(0))$")
    plt.plot(r_tt[i],r_INV_L[i],'-b',label="Relaxation: $L(p(t),q(t))-L(p(0),q(0))$")
    plt.plot(b_tt[i],b_INV_LRL[i],':',color='lime',label="Baseline: $A(p(t),q(t))-A(p(0),q(0))$")
    plt.plot(r_tt[i],r_INV_LRL[i],'-',color='lime',label="Relaxation:$A(p(t),q(t))-A(p(0),q(0))$")
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.xlabel("$t$");
    plt.yscale("symlog", linthresh=1.e-14)
    plt.yticks([-1.e-6, -1.e-10, -1.e-14, 1.e-14, 1.e-10, 1.e-6])
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.2))
plt.savefig("./Figures/Kepler_Two_Body_3_Inv_Error_Time.pdf", bbox_inches="tight")

### Compute and plot errors by different methods

In [ ]:
b_ERR = []; r_ERR = []
for i in range(len(methods)):
    # maximum norm
    b_err = [np.max(np.abs(b_uu[i][j,0:2] - u_kepler(b_tt[i][j])[0:2])) for j in np.arange(len(b_tt[i]))]
    r_err = [np.max(np.abs(r_uu[i][j,0:2] - u_kepler(r_tt[i][j])[0:2])) for j in np.arange(len(r_tt[i]))]
    b_ERR.append(b_err); r_ERR.append(r_err)

In [ ]:
lgd_box_pos = [[0.4,0.2,0.5, 0.5],[0.4,0.1,0.5, 0.5],[0.4,0.5,0.5, 0.5]]
sl1_cons_mult = [8e-6,1e-6,5e-6]; sl1_p = [1,1,1]
sl2_cons_mult = [2*1e-4,6e-8,4*1e-7]; sl2_p = [2,2,2]
y_scale_line = [1e-10,1e-8,1e-7]
shift = 1

font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 
plt.figure(figsize=(15, 4))

for i in range(len(methods)):
    plt.subplot(1,3,i+1)
    plt.plot(b_tt[i]+shift,b_ERR[i],':',color='orangered',label="Baseline")
    plt.plot(r_tt[i]+shift,r_ERR[i],'-g',label="Relaxation")
    sl_t = np.linspace(10,200,1000)

    plt.plot(sl_t,sl1_cons_mult[i]*sl_t**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl1_p[i]))
    plt.plot(sl_t,sl2_cons_mult[i]*sl_t**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl2_p[i]))
    
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel('t'); plt.ylabel('Error in q')
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=6, bbox_to_anchor=(0.5, 1.1))
plt.savefig("./Figures/Kepler_Two_Body_Err_Time.pdf", bbox_inches="tight")